In [7]:
!pip install pandas pyarrow
# или для parquet можно fastparquet:
# pip install pandas fastparquet



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [8]:
# === КОНФИГ ===

# Входной файл: .csv, .tsv, .tab, .parquet
INPUT_FILE = "sample_submission.tsv"

# Выходной JSON
OUTPUT_JSON = "data.json"

# Название столбца для ID
# Если None – query_id будет 1,2,3,... по порядку
ID_COLUMN = "app_name"          # напр. "id", "query_id" или None

# Название столбца для текста вопроса
QUESTION_COLUMN = "labels_str"    # напр. "question", "text", "prompt"


In [9]:
import json
from pathlib import Path

import pandas as pd


In [10]:
TABULAR_EXTS = {".csv", ".tsv", ".tab", ".parquet"}

def load_tabular(input_path: str) -> pd.DataFrame:
    """
    Загружает csv/tsv/parquet в pandas.DataFrame.
    """
    path = Path(input_path)
    ext = path.suffix.lower()

    if ext == ".csv":
        df = pd.read_csv(path)
    elif ext in (".tsv", ".tab"):
        df = pd.read_csv(path, sep="\t")
    elif ext == ".parquet":
        df = pd.read_parquet(path)
    else:
        raise ValueError(
            f"Неподдерживаемое расширение: {ext}. Ожидаются .csv, .tsv, .tab или .parquet"
        )

    return df


def to_rag_json(input_file: str,
                output_json: str,
                id_column: str | None,
                question_column: str):
    df = load_tabular(input_file)

    if question_column not in df.columns:
        raise KeyError(
            f"Нет колонки для текста '{question_column}'. "
            f"Доступные колонки: {list(df.columns)}"
        )

    df = df.copy()

    # ID
    if id_column is None:
        df["query_id"] = range(1, len(df) + 1)
    else:
        if id_column not in df.columns:
            raise KeyError(
                f"Нет колонки для ID '{id_column}'. "
                f"Доступные колонки: {list(df.columns)}"
            )
        df = df.rename(columns={id_column: "query_id"})

    # Оставляем только нужные поля
    df = df[["query_id", question_column]].rename(
        columns={question_column: "question"}
    )

    # Убираем пустые вопросы
    df = df.dropna(subset=["question"])

    # Приводим к строкам
    df["query_id"] = df["query_id"].astype(str)
    df["question"] = df["question"].astype(str)

    records = df.to_dict(orient="records")

    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)

    print(f"Сохранено {len(records)} записей в {output_json}")


In [11]:
to_rag_json(
    INPUT_FILE,
    OUTPUT_JSON,
    id_column=ID_COLUMN,
    question_column=QUESTION_COLUMN,
)


Сохранено 15046 записей в data.json
